# Example 3: Extract Class, Move Field, Move Method

[Extract class in the refactoring catalog.](http://refactoring.com/catalog/extractClass.html)

[Move field in the refactoring catalog](http://refactoring.com/catalog/moveField.html).

[Move method in the refactoring catalog](http://refactoring.com/catalog/moveMethod.html).

---

Say you're representing a family pet that has a name.

In [4]:
class Pet:
    def __init__(self, name):
        self.name = name

In [5]:
my_pet = Pet('Gregory the Gila Monster')
print(my_pet.name)

Gregory the Gila Monster


---

Over time this class may get more complex, like adding the pet's age.

In [6]:
class Pet:
    def __init__(self, name, age):
        self.name = name
        self.age = age

In [7]:
my_pet = Pet('Gregory the Gila Monster', 3)
print('%s is %d years old' % (my_pet.name, my_pet.age))

Gregory the Gila Monster is 3 years old


---

You may also add an action method for feeding the pet, and keeping track of how much you've fed it.

In [8]:
class Pet:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        self.treats_eaten = 0
    
    def give_treats(self, count):
        self.treats_eaten += count

In [9]:
my_pet = Pet('Gregory the Gila Monster', 3)
my_pet.give_treats(2)
print('%s ate %d treats' % (my_pet.name, my_pet.treats_eaten))

Gregory the Gila Monster ate 2 treats


---

Now we want to know what kind of pet it is, so we know how to take care of it. We add various attributes and helper methods to accomplish this.

In [10]:
class Pet:
    def __init__(self, name, age, *, has_scales=False, lays_eggs=False, drinks_milk=False):
        self.name = name
        self.age = age
        self.treats_eaten = 0
        self.has_scales = has_scales
        self.lays_eggs = lays_eggs
        self.drinks_milk = drinks_milk

    def give_treats(self, count):
        self.treats_eaten += count
    
    @property
    def needs_heat_lamp(self):
        return (
            self.has_scales and
            self.lays_eggs and
            not self.drinks_milk)

In [11]:
my_pet = Pet('Gregory the Gila Monster', 3, has_scales=True, lays_eggs=True)
print('%s needs a heat lamp? %s' % (my_pet.name, my_pet.needs_heat_lamp))

Gregory the Gila Monster needs a heat lamp? True


In [12]:
my_pet.give_treats(3)
my_pet.give_treats(2)
print('%s ate %d treats' % (my_pet.name, my_pet.treats_eaten))

Gregory the Gila Monster ate 5 treats


---

At this point, it's looking like this class may end up with too many responsibilities. We should create a new class called `Animal` that will separate the intrinsic attributes of the creature from what makes it a `Pet` (like a given name).

We do this by extracting the class `Animal` from `Pet`. We move fields like `has_scales` from the old usage to the new usage. And we use a parameter object to change the constructor of `Pet`.

In [13]:
import warnings

In [14]:
class Animal:
    def __init__(self, *, has_scales=False, lays_eggs=False, drinks_milk=False):
        self.has_scales = has_scales
        self.lays_eggs = lays_eggs
        self.drinks_milk = drinks_milk

class Pet:
    def __init__(self, name, age, animal=None, **kwargs):
        self.name = name
        self.age = age
        self.treats_eaten = 0

        if kwargs and animal is not None:
            raise TypeError('Must supply either an Animal instance or keyword arguments')

        if kwargs:
            warnings.warn('Must directly pass an Animal instance')
            animal = Animal(**kwargs)
        elif animal is None:
            raise TypeError('Must supply "animal"')
        
        self.animal = animal

    def give_treats(self, count):
        self.treats_eaten += count

    @property
    def needs_heat_lamp(self):  # Changed to reference self.animal
        return (
            self.animal.has_scales and
            self.animal.lays_eggs and
            not self.animal.drinks_milk)

    @property
    def has_scales(self):
        warnings.warn('Use animal.has_scales instead')
        return self.animal.has_scales

    @property
    def lays_eggs(self):
        warnings.warn('Use animal.lays_eggs instead')
        return self.animal.lays_eggs

    @property
    def drinks_milk(self):
        warnings.warn('Use animal.drinks_milk instead')
        return self.animal.drinks_milk

In [15]:
animal = Animal(has_scales=True, lays_eggs=True)
my_pet = Pet('Gregory the Gila Monster', 3, animal)  # New usage with no warnings
print('%s needs a heat lamp? %s' % (my_pet.name, my_pet.needs_heat_lamp))

Gregory the Gila Monster needs a heat lamp? True


In [16]:
my_pet = Pet('Gregory the Gila Monster', 3, has_scales=True, lays_eggs=True)  # Warning expected because of old usage
print('%s needs a heat lamp? %s' % (my_pet.name, my_pet.needs_heat_lamp))

Gregory the Gila Monster needs a heat lamp? True


/Users/bslatkin/projects/pycon2016/lib/python3.5/site-packages/ipykernel/__main__.py:17: UserWarning: Must directly pass an Animal instance


In [17]:
my_pet = Pet('Gregory the Gila Monster', 3)  # Error expected because parameters are missing

TypeError: Must supply "animal"

In [18]:
my_pet = Pet('Gregory the Gila Monster', 3, animal, has_scales=True)  # Error expected because of mixed usage

TypeError: Must supply either an Animal instance or keyword arguments

---

Here's the final extracted class now that all of the usage has been moved to the new version.

In [268]:
class Animal:
    def __init__(self, *, has_scales=False, lays_eggs=False, drinks_milk=False):
        self.has_scales = has_scales
        self.lays_eggs = lays_eggs
        self.drinks_milk = drinks_milk

class Pet:
    def __init__(self, name, age, animal):
        self.name = name
        self.age = age
        self.animal = animal
        self.treats_eaten = 0

    def give_treats(self, count):
        self.treats_eaten += count

    @property
    def needs_heat_lamp(self):
        return (
            self.animal.has_scales and
            self.animal.lays_eggs and
            not self.animal.drinks_milk)

In [269]:
animal = Animal(has_scales=True, lays_eggs=True)
my_pet = Pet('Gregory the Gila Monster', 3, animal)
print('%s needs a heat lamp? %s' % (my_pet.name, my_pet.needs_heat_lamp))

Gregory the Gila Monster needs a heat lamp? True


---

Now we want to move the `needs_heat_lamp` method into the `Animal` class. It's best not to refactor too many things at the same time. The point here is you can do it piecemeal and you'll existing a transitional state where part of your APIs looks good and part of it doesn't look good yet.

In [270]:
class Animal:
    def __init__(self, *, has_scales=False, lays_eggs=False, drinks_milk=False):
        self.has_scales = has_scales
        self.lays_eggs = lays_eggs
        self.drinks_milk = drinks_milk

    @property
    def needs_heat_lamp(self):
        return (
            self.has_scales and
            self.lays_eggs and
            not self.drinks_milk)
        
class Pet:
    def __init__(self, name, age, animal):
        self.name = name
        self.age = age
        self.animal = animal
        self.treats_eaten = 0

    def give_treats(self, count):
        self.treats_eaten += count

    @property
    def needs_heat_lamp(self):
        warnings.warn('Must use animal.needs_heat_lamp')
        return self.animal.needs_heat_lamp

In [271]:
animal = Animal(has_scales=True, lays_eggs=True)
my_pet = Pet('Gregory the Gila Monster', 3, animal)
print('%s needs a heat lamp? %s' % (my_pet.name, my_pet.animal.needs_heat_lamp))  # New usage works

Gregory the Gila Monster needs a heat lamp? True


In [272]:
animal = Animal(has_scales=True, lays_eggs=True)
my_pet = Pet('Gregory the Gila Monster', 3, animal)
print('%s needs a heat lamp? %s' % (my_pet.name, my_pet.needs_heat_lamp))  # Warning expected because of old usage

Gregory the Gila Monster needs a heat lamp? True


/Users/bslatkin/projects/pycon2016/lib/python3.5/site-packages/ipykernel/__main__.py:26: UserWarning: Must use animal.needs_heat_lamp


---

After fix all of the old usage of `needs_heat_lamp`, here's what the classes will look like.

In [273]:
class Animal:
    def __init__(self, *, has_scales=False, lays_eggs=False, drinks_milk=False):
        self.has_scales = has_scales
        self.lays_eggs = lays_eggs
        self.drinks_milk = drinks_milk

    @property
    def needs_heat_lamp(self):
        return (
            self.has_scales and
            self.lays_eggs and
            not self.drinks_milk)
        
class Pet:
    def __init__(self, name, age, animal):
        self.name = name
        self.age = age
        self.animal = animal
        self.treats_eaten = 0

    def give_treats(self, count):
        self.treats_eaten += count

In [274]:
animal = Animal(has_scales=True, lays_eggs=True)
my_pet = Pet('Gregory the Gila Monster', 3, animal)
print('%s needs a heat lamp? %s' % (my_pet.name, my_pet.animal.needs_heat_lamp))  # New usage, no errors

Gregory the Gila Monster needs a heat lamp? True


In [275]:
animal = Animal(has_scales=True, lays_eggs=True)
my_pet = Pet('Gregory the Gila Monster', 3, animal)
print('%s needs a heat lamp? %s' % (my_pet.name, my_pet.needs_heat_lamp))  # Error expected because it's old usage

AttributeError: 'Pet' object has no attribute 'needs_heat_lamp'

This same approach works for both methods and `@property` methods.

---

Another intrinsic property of the pet is its `age`. We should refactor this into the interior class. This is more challenging because it's a property being assigned. If you do it the same was as the other attributes, it will break.

In [276]:
animal = Animal(has_scales=True, lays_eggs=True)
my_pet = Pet('Gregory the Gila Monster', 3, animal)
my_pet.age = 5
print('%s is %d years old' % (my_pet.name, my_pet.age))

Gregory the Gila Monster is 5 years old


In [277]:
class Animal:
    def __init__(self, age=None, *, has_scales=False, lays_eggs=False, drinks_milk=False):
        if age is None:
            warnings.warn('Should specify "age" for Animal')
        self.age = age
        self.has_scales = has_scales
        self.lays_eggs = lays_eggs
        self.drinks_milk = drinks_milk

    @property
    def needs_heat_lamp(self):
        return (
            self.has_scales and
            self.lays_eggs and
            not self.drinks_milk)
        
class Pet:
    def __init__(self, name, age_or_animal, maybe_animal=None):
        self.name = name

        if maybe_animal is not None:
            warnings.warn('Should specify "age" for Animal')
            self.animal = maybe_animal
            self.animal.age = age_or_animal
        else:
            self.animal = age_or_animal

        self.treats_eaten = 0

    def give_treats(self, count):
        self.treats_eaten += count

    @property
    def age(self):
        warnings.warn('Should use animal.age')
        return self.animal.age

In [278]:
animal = Animal(has_scales=True, lays_eggs=True)  # Warning expected
my_pet = Pet('Gregory the Gila Monster', 3, animal)  # Warning expected
print('%s is %d years old' % (my_pet.name, my_pet.age))  # Warning expected

Gregory the Gila Monster is 3 years old


/Users/bslatkin/projects/pycon2016/lib/python3.5/site-packages/ipykernel/__main__.py:4: UserWarning: Should specify "age" for Animal
/Users/bslatkin/projects/pycon2016/lib/python3.5/site-packages/ipykernel/__main__.py:22: UserWarning: Should specify "age" for Animal
/Users/bslatkin/projects/pycon2016/lib/python3.5/site-packages/ipykernel/__main__.py:35: UserWarning: Should use animal.age


In [279]:
animal = Animal(3, has_scales=True, lays_eggs=True)  # No warnings for new usage
my_pet = Pet('Gregory the Gila Monster', animal)
print('%s is %d years old' % (my_pet.name, my_pet.animal.age))  # No warnings for new usage

Gregory the Gila Monster is 3 years old


---

One problem is that this middle refactoring state breaks when you try to assign to the `age` attribute.

In [280]:
my_pet.age = 5  # Error expected

AttributeError: can't set attribute

What's missing is the `@property.setter` that can handle assignment during the transition period.

In [281]:
class Animal:
    def __init__(self, age=None, *, has_scales=False, lays_eggs=False, drinks_milk=False):
        if age is None:
            warnings.warn('Should specify "age" for Animal')
        self.age = age
        self.has_scales = has_scales
        self.lays_eggs = lays_eggs
        self.drinks_milk = drinks_milk

    @property
    def needs_heat_lamp(self):
        return (
            self.has_scales and
            self.lays_eggs and
            not self.drinks_milk)
        
class Pet:
    def __init__(self, name, age_or_animal, maybe_animal=None):
        self.name = name

        if maybe_animal is not None:
            warnings.warn('Should specify "age" for Animal')
            self.animal = maybe_animal
            self.animal.age = age_or_animal
        else:
            self.animal = age_or_animal

        self.treats_eaten = 0

    def give_treats(self, count):
        self.treats_eaten += count

    @property
    def age(self):
        warnings.warn('Should use animal.age')
        return self.animal.age

    @age.setter
    def age(self, new_age):
        warnings.warn('Should assign animal.age')
        self.animal.age = new_age

In [282]:
animal = Animal(3, has_scales=True, lays_eggs=True)
my_pet = Pet('Gregory the Gila Monster', animal)
my_pet.age = 5  # Warning expected because of old usage

/Users/bslatkin/projects/pycon2016/lib/python3.5/site-packages/ipykernel/__main__.py:40: UserWarning: Should assign animal.age


---

Now we can move everything over to the new usage, and remove the old way.

In [283]:
class Animal:
    def __init__(self, age, *, has_scales=False, lays_eggs=False, drinks_milk=False):
        self.age = age
        self.has_scales = has_scales
        self.lays_eggs = lays_eggs
        self.drinks_milk = drinks_milk

    @property
    def needs_heat_lamp(self):
        return (
            self.has_scales and
            self.lays_eggs and
            not self.drinks_milk)
        
class Pet:
    def __init__(self, name, animal):
        self.name = name
        self.animal = animal
        self.treats_eaten = 0

    def give_treats(self, count):
        self.treats_eaten += count

In [284]:
animal = Animal(3, has_scales=True, lays_eggs=True)  # New usage works
my_pet = Pet('Gregory the Gila Monster', animal)
my_pet.animal.age = 5
print('%s is %d years old' % (my_pet.name, my_pet.animal.age))  # New usage works

Gregory the Gila Monster is 5 years old


---

One gotcha of extracting assignable attributes is you can still accidentally use the old assignments because Python will add an attribute to a class when it's assigned.

In [285]:
animal = Animal(3, has_scales=True, lays_eggs=True)
my_pet = Pet('Gregory the Gila Monster', animal)
my_pet.age = 5  # Old usage doesn't raise an error
print('%s is %d years old' % (my_pet.name, my_pet.animal.age))  # Prints wrong result

Gregory the Gila Monster is 3 years old


You can avoid this by leaving behind a tombstone property to prevent this type of usage. Old habits and muscle memory are hard to shake, so this can be a worthwhile investment if you're refactoring particularly commonly used components.

In [286]:
class Animal:
    def __init__(self, age, *, has_scales=False, lays_eggs=False, drinks_milk=False):
        self.age = age
        self.has_scales = has_scales
        self.lays_eggs = lays_eggs
        self.drinks_milk = drinks_milk

    @property
    def needs_heat_lamp(self):
        return (
            self.has_scales and
            self.lays_eggs and
            not self.drinks_milk)
        
class Pet:
    def __init__(self, name, animal):
        self.name = name
        self.animal = animal
        self.treats_eaten = 0

    def give_treats(self, count):
        self.treats_eaten += count

    @property
    def age(self):
        raise AttributeError('Must use animal.age')

    @age.setter
    def age(self, new_age):
        raise AttributeError('Must assign animal.age')

Now we'll get an error immediately on accidental usage of the pre-migration approach.

In [287]:
animal = Animal(3, has_scales=True, lays_eggs=True)
my_pet = Pet('Gregory the Gila Monster', animal)
my_pet.age = 5  # Old usage raises an error

AttributeError: Must assign animal.age